In [1]:
# Installs Unsloth, Xformers (Flash Attention), and other dependencies
# This usually takes 2-3 minutes on Colab
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-_8bxk1ug/unsloth_9c5485501eb44246bb115601eebf44c5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-_8bxk1ug/unsloth_9c5485501eb44246bb115601eebf44c5
  Resolved https://github.com/unslothai/unsloth.git to commit d1e312dcdc57bf020aa0f6da810226efe79cd69a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 

In [1]:
!nvidia-smi

Thu Dec  4 17:08:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
from unsloth import FastLanguageModel
import torch
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None          # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True   # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
# Load dataset from Hugging Face Hub
# Replace 'YOUR_USERNAME' with your actual HF username after running the upload script
HF_DATASET = "andrea-t94/airflow-dag-dataset"

print(f"📦 Loading dataset from Hugging Face: {HF_DATASET}")

📦 Loading dataset from Hugging Face: andrea-t94/airflow-dag-dataset


In [5]:
from datasets import load_dataset

# Load dataset from Hugging Face Hub (already split into train/eval/test)
split_dataset = load_dataset(HF_DATASET)

print(f"✓ Dataset loaded from Hugging Face Hub")
print(f"\nDataset Split Sizes:")
print(f"  Train: {len(split_dataset['train'])} samples")
print(f"  Eval:  {len(split_dataset['eval'])} samples")
print(f"  Test:  {len(split_dataset['test'])} samples")

# Dataset is already in ChatML format (messages field)
# Apply the chat template for Qwen fine-tuning
def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        # Apply the chat template - tokenizer will handle ChatML formatting
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return {"text": texts}

# Apply formatting to all splits
split_dataset = split_dataset.map(formatting_prompts_func, batched=True)

print("\n✓ Chat template applied to all splits")
print("Ready for training!")

README.md:   0%|          | 0.00/764 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

data/eval-00000-of-00001.parquet:   0%|          | 0.00/572k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/559k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7414 [00:00<?, ? examples/s]

...

✓ Dataset loaded from Hugging Face Hub

Dataset Split Sizes:
  Train: 7414 samples
  Eval:  412 samples
  Test:  412 samples


Map:   0%|          | 0/7414 [00:00<?, ? examples/s]

Map:   0%|          | 0/412 [00:00<?, ? examples/s]

Map:   0%|          | 0/412 [00:00<?, ? examples/s]


✓ Chat template applied to all splits
Ready for training!


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = split_dataset["train"],
    eval_dataset = split_dataset["eval"], # Pass the validation set here
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can set to True for speed boost, but be careful with short seqs
    args = TrainingArguments(
        per_device_train_batch_size = 2, # Increase to 4 if VRAM allows (T4 is tight)
        gradient_accumulation_steps = 4, # Simulates batch size of 8 (2 * 4)
        warmup_steps = 5,
        max_steps = 0, # Set to 0 to use num_train_epochs
        num_train_epochs = 1, # Start with 1 epoch to test time! typically 3 is good.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit", # Key for memory saving
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_strategy = "epoch", # Save checkpoint every epoch
        eval_strategy = "steps", # Check eval loss during training
        eval_steps = 50, # Evaluate every 50 steps
    ),
)

# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# START TRAINING
trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/7414 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/412 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


GPU = Tesla T4. Max memory = 14.741 GB.
1.525 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,414 | Num Epochs = 1 | Total steps = 0
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models


In [ ]:
# --------------------------------------------------------
# Step 6: Evaluate Fine-Tuned vs. Base Model on Test Set
# --------------------------------------------------------
from unsloth import FastLanguageModel
import random

# 1. Switch model to Inference Mode
FastLanguageModel.for_inference(model)

# 2. Pick random samples from the TEST set (held out data)
test_samples = split_dataset['test'].select(range(5)) # Select first 5, or use .shuffle()

print("=========================================")
print("📊 EVALUATION: Base vs. Fine-Tuned")
print("=========================================")

for i, sample in enumerate(test_samples):
    # Prepare the input prompt (without the response part)
    # We parse the original ChatML messages to extract just the user/system prompt
    # Note: Depending on your dataset structure, you might need to grab the raw 'messages'
    # Here we assume the 'text' field is already formatted, so we strip the assistant response for the prompt.
    
    # SAFER METHOD: Re-construct prompt from raw data if possible, or simple split:
    full_text = sample['text']
    # Split at the start of the assistant generation to create the prompt
    prompt = full_text.split("<|im_start|>assistant\n")[0] + "<|im_start|>assistant\n"
    ground_truth = full_text.split("<|im_start|>assistant\n")[1].replace("<|im_end|>", "")

    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    # --- A. Generate with FINE-TUNED Model (Adapters Active) ---
    # Unsloth has adapters active by default after training
    outputs_ft = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    response_ft = tokenizer.batch_decode(outputs_ft)[0].split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "")

    # --- B. Generate with BASE Model (Disable Adapters) ---
    # We use a context manager to temporarily disable LoRA adapters
    with model.disable_adapter():
        outputs_base = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
        response_base = tokenizer.batch_decode(outputs_base)[0].split("<|im_start|>assistant\n")[-1].replace("<|im_end|>", "")

    # --- Print Comparison ---
    print(f"\n🔹 TEST SAMPLE {i+1}")
    print(f"📝 PROMPT: {prompt[:100]}... (truncated)") 
    print("-" * 40)
    print(f"🧠 BASE MODEL:\n{response_base.strip()[:300]}...") # Truncated for display
    print("-" * 20)
    print(f"🚀 FINE-TUNED:\n{response_ft.strip()[:300]}...")
    print("-" * 40)
    print(f"✅ GROUND TRUTH:\n{ground_truth.strip()[:300]}...")
    print("=========================================")

In [ ]:
# 1. Login to Hugging Face
from huggingface_hub import login
login()

repo_name = "andrea-t94/qwen2.5-1.5b-airflow-instruct"

# --- A. Push LoRA Adapters (Most Portable & Lightweight) ---
# This allows anyone to load your fine-tune on top of Qwen-Base
model.push_to_hub(repo_name, token=True)
tokenizer.push_to_hub(repo_name, token=True)
print("✅ Saved LoRA Adapters (Source)")

# --- B. Push Merged FP16 Model (The "Standard" Standalone) ---
# This merges the adapters into the base model and saves as full precision (safetensors).
# Use this if you want to deploy to vLLM later or re-quantize to AWQ/GPTQ.
model.push_to_hub_merged(
    repo_name + "-merged", # I recommend a separate repo or branch for the full merge
    tokenizer,
    save_method = "merged_16bit",
    token = True
)
print("✅ Saved Merged FP16 Model (Standard)")

# --- C. Push GGUF (For your M1 Mac) ---
# This is the "build artifact" you need right now.
model.push_to_hub_gguf(
    repo_name, # Can live in the same repo as adapters
    tokenizer,
    quantization_method = "q8_0",
    token = True
)
print("✅ Saved GGUF (Mac Ready)")